In [3]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

In [12]:
# Transcribe the data into a list of dictionaries
data = [
    {"Tenured": "No", "Area Name": "COMBINED", "Sample": 655, "Metric": "Adherence", "WFH Proportion": 0.828},
    {"Tenured": "No", "Area Name": "COMBINED", "Sample": 655, "Metric": "Consults", "WFH Proportion": 0.182},
    {"Tenured": "No", "Area Name": "COMBINED", "Sample": 655, "Metric": "OSAT", "WFH Proportion": 0.470},
    {"Tenured": "No", "Area Name": "COMBINED", "Sample": 655, "Metric": "Transfers", "WFH Proportion": 0.182},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Adherence", "WFH Proportion": 0.755},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Consults", "WFH Proportion": 0.153},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "OSAT", "WFH Proportion": 0.552},
    {"Tenured": "No", "Area Name": "Chat", "Sample": 4, "Metric": "Transfers", "WFH Proportion": 0.129},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Adherence", "WFH Proportion": 0.912},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Consults", "WFH Proportion": 0.206},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "OSAT", "WFH Proportion": 0.472},
    {"Tenured": "No", "Area Name": "Client Banking Services", "Sample": 46, "Metric": "Transfers", "WFH Proportion": 0.148},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Adherence", "WFH Proportion": 0.836},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Consults", "WFH Proportion": 0.178},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "OSAT", "WFH Proportion": 0.468},
    {"Tenured": "No", "Area Name": "Core Service", "Sample": 594, "Metric": "Transfers", "WFH Proportion": 0.188},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Adherence", "WFH Proportion": 0.888},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Consults", "WFH Proportion": 0.104},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "OSAT", "WFH Proportion": 1.000},
    {"Tenured": "No", "Area Name": "Tier 2 Support", "Sample": 5, "Metric": "Transfers", "WFH Proportion": 0.241},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Adherence", "WFH Proportion": 0.822},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Consults", "WFH Proportion": 0.266},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "OSAT", "WFH Proportion": 0.571},
    {"Tenured": "No", "Area Name": "Trader Service", "Sample": 6, "Metric": "Transfers", "WFH Proportion": 0.273},
]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Adding some hypothetical Office Proportion data
np.random.seed(42)  # For reproducibility
df["Office Proportion"] = np.random.rand(len(df))

# Set the alpha level (e.g., 0.05 for a 95% confidence level)
alpha = 0.05

# Perform Z-tests for each row
results = []

for _, row in df.iterrows():
    count = np.array([row["WFH Proportion"] * row["Sample"], row["Office Proportion"] * row["Sample"]])
    nobs = np.array([row["Sample"], row["Sample"]])
    
    z_score, p_value = proportions_ztest(count, nobs)
    
    results.append({
        "Tenured": row["Tenured"],
        "Area Name": row["Area Name"],
        "Sample": row["Sample"],
        "Metric": row["Metric"],
        "WFH Proportion": row["WFH Proportion"],
        "Office Proportion": row["Office Proportion"],
        "Z-Statistic": z_score,
        "P-Value": p_value,
        "Significant": p_value < alpha  # Check if the p-value is less than the alpha level
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Reorder the columns
df_results = df_results[["Tenured", "Area Name", "Sample", "Metric", "WFH Proportion", "Office Proportion", "Z-Statistic", "P-Value", "Significant"]]

# Print the DataFrame
print(df_results)

# Optionally, save the results to an Excel file
df_results.to_excel('Z_Test_Results.xlsx', index=False)

   Tenured                Area Name  Sample     Metric  WFH Proportion  \
0       No                 COMBINED     655  Adherence           0.828   
1       No                 COMBINED     655   Consults           0.182   
2       No                 COMBINED     655       OSAT           0.470   
3       No                 COMBINED     655  Transfers           0.182   
4       No                     Chat       4  Adherence           0.755   
5       No                     Chat       4   Consults           0.153   
6       No                     Chat       4       OSAT           0.552   
7       No                     Chat       4  Transfers           0.129   
8       No  Client Banking Services      46  Adherence           0.912   
9       No  Client Banking Services      46   Consults           0.206   
10      No  Client Banking Services      46       OSAT           0.472   
11      No  Client Banking Services      46  Transfers           0.148   
12      No             Core Service   

In [10]:
def perform_t_tests(df, metrics):
    # Get unique values for 'area' and 'tenured'
    areas = df['area'].unique()
    tenured_statuses = df['tenured'].unique()
    
    # Initialize a list to store results
    results = []

    for tenured in tenured_statuses:
        for area in areas:
            for metric in metrics:
                # Subset data based on 'tenured' and 'area'
                subset = df[(df['tenured'] == tenured) & (df['area'] == area)]
                # Separate data into 'HOME' and 'OFFICE' groups
                wfh_data = subset[subset['day_type'] == 'HOME'][metric]
                office_data = subset[subset['day_type'] == 'OFFICE'][metric]
                
                # Perform T-Test
                t_stat, p_value = ttest_ind(wfh_data, office_data, equal_var=False, nan_policy='omit')
                
                # Calculate means and sample sizes
                wfh_mean = wfh_data.mean()
                office_mean = office_data.mean()
                wfh_sample_size = wfh_data.count()
                office_sample_size = office_data.count()
                
                # Append results to the list
                results.append({
                    'Tenured': tenured,
                    'Area': area,
                    'Metric': metric,
                    'WFH Mean': wfh_mean,
                    'Office Mean': office_mean,
                    'WFH Sample Size': wfh_sample_size,
                    'Office Sample Size': office_sample_size,
                    'T-Statistic': t_stat,
                    'P-Value': round(p_value, 5)
                })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

In [14]:
import pandas as pd
from scipy.stats import ttest_ind

# Define the function to perform T-Tests
def perform_t_tests(df, metrics):
    # Get unique values for 'area' and 'tenured'
    areas = df['area'].unique()
    tenured_statuses = df['tenured'].unique()
    
    # Initialize a list to store results
    results = []

    for tenured in tenured_statuses:
        for area in areas:
            for metric in metrics:
                # Subset data based on 'area' and 'tenured'
                subset = df[(df['tenured'] == tenured) & (df['area'] == area)]
                # Separate data into 'HOME' and 'OFFICE' groups
                wfh_data = subset[subset['day_type'] == 'HOME'][metric]
                office_data = subset[subset['day_type'] == 'OFFICE'][metric]
                
                # Perform T-Test
                t_stat, p_value = ttest_ind(wfh_data, office_data, equal_var=False, nan_policy='omit')
                
                # Calculate means and sample sizes
                wfh_mean = wfh_data.mean()
                office_mean = office_data.mean()
                wfh_sample_size = wfh_data.count()
                office_sample_size = office_data.count()
                
                # Append results to the list
                results.append({
                    'Tenured': tenured,
                    'Area': area,
                    'Metric': metric,
                    'WFH Mean': wfh_mean,
                    'Office Mean': office_mean,
                    'WFH Sample Size': wfh_sample_size,
                    'Office Sample Size': office_sample_size,
                    'T-Statistic': t_stat,
                    'P-Value': round(p_value, 5)
                })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    return results_df

# Example usage with your dataset
# Assuming 'result_df' is your DataFrame loaded with your data
metrics = ['avg_interaction_count', 'aht', 'avg_productivity']

# Perform the T-Tests and get the results
t_test_results = perform_t_tests(result_df, metrics)

# Save the results to an Excel file
t_test_results.to_excel('/mnt/data/Results_T_Test.xlsx', index=False)

# Display the first few rows of the results
print(t_test_results.head(5))

NameError: name 'result_df' is not defined